## A Simulated IOT Federated Learning Framework for Forest Fire Prediction

Forest fire ignition prediction is essential to the safety of communities and the proper allocation of fire fighting resources. This notebook simulates a network of clients that collect local meteorological data. Each client participates in a federated learning system where a client trains a machine learning model to classify whether the area will have a forest fire ignition. The model weights of each client are shared with a central server that averages the weights and shares them back to the clients.

This notebook demonstrates the three machine learning types shown below.

|<center>name|<center>server|<center>clients|<center>training|<center>training data|<center>evaluation type|<center>evaluation|<center>evaluation data|
|---|---|---|---|---|---|---|---|
|<center>classic machine learning|<center>yes|<center>no|<center>on server|<center>on server|<center>centralized|<center>on server|<center>on server|
|<center>federated machine learning (Central Eval)|<center>yes|<center>yes|<center>on clients|<center>on clients|<center>centralized|<center>on server|<center>on server|
|<center>federated machine learning (Federated Eval)|<center>yes|<center>yes|<center>on clients|<center>on clients|<center>distributed|<center>on clients|<center>on clients|


The data used is from the paper: *Framework for Creating Forest Fire Ignition Prediction Datasets.* Each row represents meteorological data at a geographical location at a specific time. TODO: Add table example.

Much of the code used in this notebook is based on the Flower code examples located [here](https://github.com/adap/flower/tree/main/examples) and the Keras timeseries tutorials located [here](https://keras.io/examples/timeseries/).
The code below is very much a work in progress. 

In [ ]:
#if this file is being used in colab set to 1 otherwise 0
using_colab = 1

In [ ]:
if (using_colab == 1):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if (using_colab ==1):
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
#load libraries
import math
import os
import glob
import gc
import datetime
import typing
from typing import List
from typing import Tuple
from typing import Dict
from typing import Optional
import random
import tempfile
from tables.file import File

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Model

In [ ]:
if (using_colab == 1):
    !pip install -q flwr[simulation] pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.

In [ ]:
import flwr as fl
from flwr.common import Metrics

In [ ]:
#overall environment settings

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Training on GPU or CPU?
#tf.config.set_visible_devices([], 'CPU')
print(
    f"Training on {'GPU' if tf.config.get_visible_devices('GPU') else 'CPU'} using TensorFlow {tf.__version__} and Flower {fl.__version__}"
)

Training on CPU using TensorFlow 2.12.0 and Flower 1.4.0


In [ ]:
#global variables

ml_type = 0 # classic ML = 0, federated ML w/ centralized evaluation = 1, federated ML w/ federated eval = 2

federated_path = "<path to your client datasets>" 
centralized_path = "<path to your server dataset"
results_path = "<path to where you want to store results>"


cid = str(0) # preliminary client id
log_dir = "<path to where you want to store Tensorflow logs>" + cid + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
# log_dir = "./logs/fit/" + cid + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

downsample_test_set = 0 # 0 if test set not downsampled, 1 otherwise

# TODO: Add the below to a client config file
num_rounds = 10
epochs = 200 # the number of epochs classic machine learning will use
fl_epochs = 20
sequence_len = 120 # 5 days * 24 hours
past_len = sequence_len
future_len = 24 # 1*24 hours
sampling_rate = 1 # in time series conversion use every row (hour) in loaded datasets
sequence_stride = 1 # in time series conversion each series is this far apart
fraction_fit = 1.0 # fraction of clients to use per training round
fraction_evaluate = 0.4 # fraction of clients to use per evaluation round

#create an array for results
np.set_printoptions(suppress=True)
results = np.empty([13,3])

## Data loading functions

In [ ]:
def normalize_data(x):
    """Normalizes the data of an array by column. 
    Shifts and scales inputs into a distribution centered around 0 
    with standard deviation 1.

    Parameters
    ----------
    x: NDarray 
        An array of feature values.
        
    Returns
    -------
    features_normalized : NDarray
        The original array, but normalized.
    """
    data = x
    layer = layers.Normalization()
    layer.adapt(data)
    features_normalized = layer(data)
    return features_normalized

In [ ]:
def mask_create(x):
    """Finds the class count of the input array and creates a mask that can be used
    to randomly downsample an array of labels so that the number of 
    negative labels = the number of positive labels. 

    Parameters
    ----------
    x: NDarray 
        An array of feature values.
        
    Returns
    -------
    features_normalized : NDarray
        A masked version of the input array.
    """
    mask_length = x.shape[0]
    mask = tf.reshape(x, [mask_length])
    y, idx, class_count = tf.unique_with_counts(mask)
    ignition_count = tf.get_static_value(class_count[1])
    mask = mask.numpy()
    count = 0 
    while count < ignition_count:
        #rand_num = random.randint(0,mask_length)
        rand_num = random.randint(1, mask_length-1)
        if (mask[rand_num] == 0):
            mask[rand_num] = 1
            count += 1
    return mask

In [ ]:
def load_datasets(path: str):
    """Loads all the csv datasets in a folder.
    The loaded data is divided into train, validation, and test sets.
    The data is turned into time series data
    All the data is normalized.
    Train and validation datasets are downsampled.
    TODO: divide this function into smaller functions

    Parameters
    ----------
    path: string 
        The path to the dataset folder.
        
    Returns
    -------
    train_x, train_y, val_x, val_y, test_x, test_y : NDarrays
        A masked version of the input array.
    """

    train_x = []
    train_y = []
    val_x = []
    val_y = []
    test_x = []
    test_y = []

    #load data
    for filename in glob.glob(os.path.join(path, '*.csv')):
        print("\nnow reading " + filename + "\n")
        #read file
        df = pd.read_csv(filename, index_col=[0])
        
        df_train = df[(df['year'] < 2001)]
        df_val = df[(df['year'] > 2001) & (df['year'] < 2012)]
        df_test = df[(df['year'] >= 2012)]
        
        features = ['stl2', 't2m', 'stl1', 'stl3', 'skt', 'swvl1', 'd2m', 'swvl2']
        train_features = df_train[features]
        train_labels = df_train[["ignition"]]
        val_features = df_val[features]
        val_labels = df_val[["ignition"]]
        test_features = df_test[features]
        test_labels = df_test[["ignition"]]
        #convert to numpy
        train_features = train_features.values
        val_features = val_features.values
        test_features = test_features.values

        #normalize
        train_features_normalize = normalize_data(train_features)
        val_features_normalize = normalize_data(val_features)
        test_features_normalize = normalize_data(test_features)
        
        #we want to predict at a future point
        #so we clip the length of the features plus the hours till the future point
        start = past_len + future_len
        train_labels = train_labels.iloc[start:].values
        val_labels = val_labels.iloc[start:].values
        test_labels = test_labels.iloc[start:].values
        
        batch_size = 107856 #factor of 5136 (321 * 16)
        #convert to time series data
        train_dataset = keras.utils.timeseries_dataset_from_array(
            train_features_normalize,
            train_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        val_dataset = keras.utils.timeseries_dataset_from_array(
            val_features_normalize,
            val_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        test_dataset = keras.utils.timeseries_dataset_from_array(
            test_features_normalize,
            test_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)
        
        #for bookkeeping print out the shapes of the datasets
        for train_features, train_labels in train_dataset:
            print("train_dataset features shape:", train_features.shape)
            print("targets_dataset labels shape:", train_labels.shape)
            break

        for val_features, val_labels in val_dataset:
            print("\nval_dataset features shape:", val_features.shape)
            print("val_dataset labels shape:", val_labels.shape)
            break

        for test_features, test_labels in test_dataset:
            print("\ntest_dataset features shape:", test_features.shape)
            print("test_dataset labels shape:", test_labels.shape)
            break
       
        # randomly downsample the data using masks
        train_mask = mask_create(train_labels)
        train_features_masked = tf.boolean_mask(train_features, train_mask)
        train_labels_masked = tf.boolean_mask(train_labels, train_mask)
        
        val_mask = mask_create(val_labels)
        val_features_masked = tf.boolean_mask(val_features, val_mask)
        val_labels_masked = tf.boolean_mask(val_labels, val_mask)
        
        test_mask = mask_create(test_labels)
        test_features_masked = tf.boolean_mask(test_features, test_mask)
        test_labels_masked = tf.boolean_mask(test_labels, test_mask)
        
        train_x.append(train_features_masked)
        train_y.append(train_labels_masked)
        val_x.append(val_features_masked)
        val_y.append(val_labels_masked)
        if (downsample_test_set == 1):
            test_x.append(test_features_masked)
            test_y.append(test_labels_masked)
        else:
            test_x.append(test_features)
            test_y.append(test_labels)
        
    print("\nDone loading data.\n")
    return train_x, train_y, val_x, val_y, test_x, test_y 



In [ ]:
def get_value_count(x):
    """A helper function that returns the count of class labels.

    Parameters
    ----------
    x: NDArray 
        An array with class labels.
        
    Returns
    -------
    non_ignition_count, ignition_count : int
        The counts of the ignition class.
    """
    length = x[0].shape[0]
    x = tf.reshape(x, [length])
    y, idx, class_count = tf.unique_with_counts(x)
    non_ignition_count = tf.get_static_value(class_count[0])
    ignition_count = tf.get_static_value(class_count[1])
    return non_ignition_count, ignition_count
    

## Model and Metrics

In [ ]:
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
]

def make_model(metrics=METRICS):
    inputs = keras.Input(shape=(sequence_len, trainloaders_x[0].shape[2]))
    x = layers.LSTM(8, activation='sigmoid')(inputs)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)
        
    return model


## Federated Learning Functions

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val, x_test, y_test, tb_callback) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val
        self.x_test, self.y_test = x_test, y_test
        self.tb_callback = tb_callback

    def get_parameters(self, config):
        return self.model.get_weights()

    # presetting config allows us to use FlowerClient with classic ml.
    def fit(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        print("in fit")
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=epochs, verbose=2, callbacks=self.tb_callback, validation_data=(self.x_val, self.y_val),)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        self.model.set_weights(parameters)
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc  = self.model.evaluate(self.x_test, self.y_test, verbose=2)
        return loss, len(self.x_val), {"tp": tp,
                                       "fp": fp,
                                       "tn": tn,
                                       "fn": fn,
                                       "accuracy": accuracy,
                                       "precision": precision,
                                       "recall": recall,
                                       "auc": auc,
                                       "prc": prc
                                      }

    # this method allows metrics to be passed when using a single server with no clients
    def evaluate_single(self, x_test, y_test):
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc = self.model.evaluate(x_test, y_test, verbose=2)
        return loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc

In [ ]:
def client_fn(cid: str) -> fl.client.Client:

    print("\nThis is client: ", cid)

    x_train_cid = trainloaders_x[int(cid)]
    y_train_cid = trainloaders_y[int(cid)]
    x_val_cid = valloaders_x[int(cid)]
    y_val_cid = valloaders_y[int(cid)]
    x_test_cid = testloaders_x[int(cid)]
    y_test_cid = testloaders_y[int(cid)]

    print("Loaded data for client: ", cid, "\n")

    METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
        #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
    ]

    def make_model(metrics=METRICS):
        inputs = keras.Input(shape=(sequence_len, x_train_cid.shape[2]))
        x = layers.LSTM(8, activation='sigmoid')(inputs)
        x = layers.Flatten()(x)
        outputs = layers.Dense(1, activation="sigmoid")(x)
        model = keras.Model(inputs, outputs)

        model.compile(
            optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics)
        
        return model
    
    print("Making model: ", cid)
    model = make_model()
    
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    tb_callback = keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)

    # Create and return client
    print("\nClient CID: " + str(cid) + " is done.\n")
    return FlowerClient(model, x_train_cid, y_train_cid, x_val_cid, y_val_cid, x_test_cid, y_test_cid, tensorboard)

In [ ]:
# TODO: this function isn't working correctly, have a look at it
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    tps = [num_examples * m["tp"] for num_examples, m in metrics]
    fps = [num_examples * m["fp"] for num_examples, m in metrics]
    tns = [num_examples * m["tn"] for num_examples, m in metrics]
    fns = [num_examples * m["fn"] for num_examples, m in metrics]
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    precisions = [num_examples * m["precision"] for num_examples, m in metrics]
    recalls = [num_examples * m["recall"] for num_examples, m in metrics]
    aucs = [num_examples * m["auc"] for num_examples, m in metrics]
    prcs = [num_examples * m["prc"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"tp": sum(tps) / sum(examples),
            "fp": sum(fps) / sum(examples),
            "tn": sum(tns) / sum(examples),
            "fn": sum(fns) / sum(examples),
            "accuracy": sum(accuracies) / sum(examples),
            "precision": sum(precisions) / sum(examples),
            "recall": sum(recalls) / sum(examples),
            "auc": sum(aucs) / sum(examples),
            "prc": sum(prcs) / sum(examples)
           }

## Launch classic machine learning

In [ ]:
# load the dataset for centralized evaluation (for classic ml training and testing) 
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(centralized_path)


now reading /content/drive/MyDrive/Colab Notebooks/FF/data/01_clients/dly_avg_1of1_50.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

Done loading data.



In [ ]:
# save copy for federated centralized evaluation
central_testloaders_x = testloaders_x.copy()
central_testloaders_y = testloaders_y.copy()

In [ ]:
count = get_value_count(trainloaders_y)
print("Train set nonignitions and ignitions are:", count)
count = get_value_count(valloaders_y)
print("Validation set nonignitions and ignitions are:", count)
count = get_value_count(testloaders_y)
print("Test set nonignitions and ignitions are:", count)
count = get_value_count(central_testloaders_y)
print("Central test set nonignitions and ignitions are:", count)

Train set nonignitions and ignitions are: (2811, 2811)
Validation set nonignitions and ignitions are: (2534, 2534)
Test set nonignitions and ignitions are: (44719, 1361)
Central test set nonignitions and ignitions are: (44719, 1361)


In [ ]:
# create an initial model to prime flower client
model = make_model()

In [ ]:
# train single server classic ml
# create a flower client that represents a classic ml single server
central_server_model = client_fn(str(0))
# set parameters for classic_ml so it can use single flower client function
parameters = model.get_weights()
# run fit
history = central_server_model.fit(parameters)


This is client:  0
Loaded data for client:  0 

Making model:  0

Client CID: 0 is done.

in fit
Epoch 1/200
176/176 - 12s - loss: 0.6433 - tp: 1715.0000 - fp: 919.0000 - tn: 1892.0000 - fn: 1096.0000 - accuracy: 0.6416 - precision: 0.6511 - recall: 0.6101 - auc: 0.6883 - prc: 0.6726 - val_loss: 0.6165 - val_tp: 1827.0000 - val_fp: 871.0000 - val_tn: 1663.0000 - val_fn: 707.0000 - val_accuracy: 0.6886 - val_precision: 0.6772 - val_recall: 0.7210 - val_auc: 0.7230 - val_prc: 0.6863 - 12s/epoch - 69ms/step
Epoch 2/200
176/176 - 8s - loss: 0.6239 - tp: 1912.0000 - fp: 1000.0000 - tn: 1811.0000 - fn: 899.0000 - accuracy: 0.6622 - precision: 0.6566 - recall: 0.6802 - auc: 0.7138 - prc: 0.7102 - val_loss: 0.6074 - val_tp: 1784.0000 - val_fp: 810.0000 - val_tn: 1724.0000 - val_fn: 750.0000 - val_accuracy: 0.6922 - val_precision: 0.6877 - val_recall: 0.7040 - val_auc: 0.7339 - val_prc: 0.7083 - 8s/epoch - 45ms/step
Epoch 3/200
176/176 - 7s - loss: 0.6184 - tp: 1870.0000 - fp: 923.0000 - tn: 1

In [ ]:
#evaluate single server classic ml
loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc = central_server_model.evaluate_single(central_testloaders_x[0], central_testloaders_y[0])

1440/1440 - 13s - loss: 0.5602 - tp: 1026.0000 - fp: 13327.0000 - tn: 31392.0000 - fn: 335.0000 - accuracy: 0.7035 - precision: 0.0715 - recall: 0.7539 - auc: 0.8069 - prc: 0.1175 - 13s/epoch - 9ms/step


In [ ]:
print("Single server classic ml evaluation\n\
\ttp:\t%d\n\
\tfp:\t%d\n\
\ttn:\t%d\n\
\tfn:\t%d\n\n\
\tloss:\t%f\n\
\tacc:\t%f\n\
\tprec:\t%f\n\
\trec:\t%f\n\
\tauc:\t%f\n\
\tprc:\t%f\
" % (tp,fp,tn,fn,loss,accuracy,precision,recall,auc,prc))

Single server classic ml evaluation
	tp:	1026
	fp:	13327
	tn:	31392
	fn:	335

	loss:	0.560181
	acc:	0.703516
	prec:	0.071483
	rec:	0.753857
	auc:	0.806898
	prc:	0.117511


In [ ]:
#write to results array
results[0] = tp
results[1][0] = fp
results[2][0] = tn
results[3][0] = fn
results[4][0] = accuracy
results[5][0] = precision
results[6][0] = recall
results[7][0] = auc
results[8][0] = prc
results[9][0] = epochs
results[10][0] = 1
results[11][0] = 1
results[12][0] = 1

In [ ]:
#remove data from memory that isn't needed
for_removal = [trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y]
del(for_removal)
del(model)
del(central_server_model)
gc.collect()

2264

## Federated machine learning

The Flower federated learning framework is used from here down. More info can be found at https://flower.dev

In [ ]:
#TODO: there is a random out-of-bounds error with masking, for now if it occurs run this cell again
#load the dataset for federated learning
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(federated_path)


now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/121.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/120.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/119.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (460

In [ ]:
NUM_CLIENTS = 24
epochs = fl_epochs

#client_resources = {"num_cpus": 2}
#if tf.config.get_visible_devices("GPU"):
 #   client_resources["num_gpus"] = 1

def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    """Centralized evaluation function"""
    model = make_model()
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
    #model.build(input_shape=(BATCH_SIZE, 28, 28, 1))
    model.set_weights(parameters)
#    loss, accuracy = model.evaluate(central_testloaders_x, central_testloaders_y, batch_size=32, verbose=0)
#    return loss, {"accuracy": accuracy}
    loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc  = model.evaluate(central_testloaders_x[0], central_testloaders_y[0], verbose=2)
    return loss, {"tp": tp,
                  "fp": fp,
                  "tn": tn,
                  "fn": fn,
                  "accuracy": accuracy,
                  "precision": precision,
                  "recall": recall,
                  "auc": auc,
                  "prc": prc
                 }


strategy = fl.server.strategy.FedAvg(
    fraction_fit=fraction_fit,  # Sample 10% of available clients for training
    fraction_evaluate=fraction_evaluate,  # Sample 5% of available clients for evaluation
    min_fit_clients=7,  # Never sample less than 10 clients for training
    min_evaluate_clients=6,  # Never sample less than 5 clients for evaluation
    min_available_clients=7,  # Wait until at least 75 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=evaluate
)

# Start simulation
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    #client_resources=client_resources,
)

INFO flwr 2023-06-07 22:23:10,057 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-07 22:23:13,121	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-06-07 22:23:14,522 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 32470332212.0, 'node:172.28.0.12': 1.0, 'CPU': 8.0, 'object_store_memory': 16235166105.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 32470332212.0, 'node:172.28.0.12': 1.0, 'CPU': 8.0, 'object_store_memory': 16235166105.0}
INFO flwr 2023-06-07 22:23:14,526 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-07 22:23:14,529 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
INFO flwr 2023-06-07 22:23:20,056 | server.py:277 | Rece

(launch_and_get_parameters pid=18781) 
(launch_and_get_parameters pid=18781) This is client:  16
(launch_and_get_parameters pid=18781) Loaded data for client:  16 
(launch_and_get_parameters pid=18781) 
(launch_and_get_parameters pid=18781) Making model:  16
(launch_and_get_parameters pid=18781) 
(launch_and_get_parameters pid=18781) Client CID: 16 is done.
(launch_and_get_parameters pid=18781) 
1440/1440 - 14s - loss: 0.7040 - tp: 1101.0000 - fp: 22860.0000 - tn: 21859.0000 - fn: 260.0000 - accuracy: 0.4983 - precision: 0.0459 - recall: 0.8090 - auc: 0.7169 - prc: 0.0660 - 14s/epoch - 9ms/step


INFO flwr 2023-06-07 22:23:33,895 | server.py:91 | initial parameters (loss, other metrics): 0.7040122151374817, {'tp': 1101.0, 'fp': 22860.0, 'tn': 21859.0, 'fn': 260.0, 'accuracy': 0.4982638955116272, 'precision': 0.04594966769218445, 'recall': 0.8089640140533447, 'auc': 0.7168745994567871, 'prc': 0.06596772372722626}
INFO:flwr:initial parameters (loss, other metrics): 0.7040122151374817, {'tp': 1101.0, 'fp': 22860.0, 'tn': 21859.0, 'fn': 260.0, 'accuracy': 0.4982638955116272, 'precision': 0.04594966769218445, 'recall': 0.8089640140533447, 'auc': 0.7168745994567871, 'prc': 0.06596772372722626}
INFO flwr 2023-06-07 22:23:33,900 | server.py:101 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2023-06-07 22:23:33,903 | server.py:218 | fit_round 1: strategy sampled 24 clients (out of 24)
DEBUG:flwr:fit_round 1: strategy sampled 24 clients (out of 24)


(launch_and_fit pid=18781) 
(launch_and_fit pid=18781) This is client:  19
(launch_and_fit pid=18781) Loaded data for client:  19 
(launch_and_fit pid=18781) 
(launch_and_fit pid=18781) Making model:  19
(launch_and_fit pid=18781) 
(launch_and_fit pid=18781) Client CID: 19 is done.
(launch_and_fit pid=18781) 
(launch_and_fit pid=18781) in fit
(launch_and_fit pid=18781) Epoch 1/20
(launch_and_fit pid=18781) 52/52 - 7s - loss: 0.6500 - tp: 615.0000 - fp: 387.0000 - tn: 440.0000 - fn: 212.0000 - accuracy: 0.6378 - precision: 0.6138 - recall: 0.7437 - auc: 0.7089 - prc: 0.7136 - val_loss: 0.6357 - val_tp: 528.0000 - val_fp: 304.0000 - val_tn: 358.0000 - val_fn: 134.0000 - val_accuracy: 0.6692 - val_precision: 0.6346 - val_recall: 0.7976 - val_auc: 0.7214 - val_prc: 0.7040 - 7s/epoch - 143ms/step
(launch_and_fit pid=18781) Epoch 2/20
(launch_and_fit pid=18776) 
(launch_and_fit pid=18776) This is client:  9
(launch_and_fit pid=18776) Loaded data for client:  9 
(launch_and_fit pid=18776) 
(l

(raylet) Spilled 4322 MiB, 1 objects, write throughput 102 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


...

(launch_and_fit pid=18776) 60/60 - 4s - loss: 0.6117 - tp: 621.0000 - fp: 294.0000 - tn: 656.0000 - fn: 329.0000 - accuracy: 0.6721 - precision: 0.6787 - recall: 0.6537 - auc: 0.7328 - prc: 0.7205 - val_loss: 0.6024 - val_tp: 462.0000 - val_fp: 220.0000 - val_tn: 445.0000 - val_fn: 203.0000 - val_accuracy: 0.6820 - val_precision: 0.6774 - val_recall: 0.6947 - val_auc: 0.7423 - val_prc: 0.7293 - 4s/epoch - 63ms/step [repeated 4x across cluster]
(launch_and_fit pid=18776) Epoch 20/20 [repeated 4x across cluster]
(launch_and_fit pid=18776) 
(launch_and_fit pid=18776) This is client:  18
(launch_and_fit pid=18776) Loaded data for client:  18 
(launch_and_fit pid=18776) 
(launch_and_fit pid=18776) Making model:  18
(launch_and_fit pid=18776) 
(launch_and_fit pid=18776) Client CID: 18 is done.
(launch_and_fit pid=18776) 
(launch_and_fit pid=18776) in fit
(launch_and_fit pid=18781) 52/52 - 3s - loss: 0.5561 - tp: 552.0000 - fp: 187.0000 - tn: 640.0000 - fn: 275.0000 - accuracy: 0.7207 - preci

DEBUG flwr 2023-06-07 23:13:49,745 | server.py:232 | fit_round 3 received 24 results and 0 failures
DEBUG:flwr:fit_round 3 received 24 results and 0 failures


1440/1440 - 14s - loss: 0.5289 - tp: 3777.0000 - fp: 56334.0000 - tn: 122542.0000 - fn: 1667.0000 - accuracy: 0.6853 - precision: 0.0628 - recall: 0.6938 - auc: 0.7480 - prc: 0.0835 - 14s/epoch - 10ms/step


INFO flwr 2023-06-07 23:14:03,911 | server.py:119 | fit progress: (3, 0.5289475917816162, {'tp': 3777.0, 'fp': 56334.0, 'tn': 122542.0, 'fn': 1667.0, 'accuracy': 0.6853244304656982, 'precision': 0.0628337562084198, 'recall': 0.6937913298606873, 'auc': 0.7479845285415649, 'prc': 0.08353571593761444}, 3030.0085402880004)
INFO:flwr:fit progress: (3, 0.5289475917816162, {'tp': 3777.0, 'fp': 56334.0, 'tn': 122542.0, 'fn': 1667.0, 'accuracy': 0.6853244304656982, 'precision': 0.0628337562084198, 'recall': 0.6937913298606873, 'auc': 0.7479845285415649, 'prc': 0.08353571593761444}, 3030.0085402880004)
DEBUG flwr 2023-06-07 23:14:03,916 | server.py:168 | evaluate_round 3: strategy sampled 9 clients (out of 24)
DEBUG:flwr:evaluate_round 3: strategy sampled 9 clients (out of 24)


(launch_and_evaluate pid=18781) 
(launch_and_evaluate pid=18781) This is client:  16
(launch_and_evaluate pid=18781) Loaded data for client:  16 
(launch_and_evaluate pid=18781) 
(launch_and_evaluate pid=18781) Making model:  16
(launch_and_fit pid=18776) 53/53 - 3s - loss: 0.6146 - tp: 494.0000 - fp: 227.0000 - tn: 612.0000 - fn: 345.0000 - accuracy: 0.6591 - precision: 0.6852 - recall: 0.5888 - auc: 0.7276 - prc: 0.7215 - val_loss: 0.6012 - val_tp: 442.0000 - val_fp: 200.0000 - val_tn: 485.0000 - val_fn: 243.0000 - val_accuracy: 0.6766 - val_precision: 0.6885 - val_recall: 0.6453 - val_auc: 0.7446 - val_prc: 0.7373 - 3s/epoch - 47ms/step [repeated 2x across cluster]
(launch_and_fit pid=18776) Making model:  16
(launch_and_evaluate pid=18781) Client CID: 16 is done.
(launch_and_evaluate pid=18776) 1440/1440 - 21s - loss: 0.5393 - tp: 193.0000 - fp: 10983.0000 - tn: 34820.0000 - fn: 84.0000 - accuracy: 0.7598 - precision: 0.0173 - recall: 0.6968 - auc: 0.7863 - prc: 0.0257 - 21s/epoch 

In [ ]:
#TODO: This whole cell needs to be cleaned up
#TODO: The weighted average function is not working correctly
# Write out the results of evaluation.

count = 0
for key in (history.metrics_distributed):
    results[count][1] = history.metrics_distributed[key][(num_rounds-1):][0][1]
    count += 1
count = 0
results[9][1] = fl_epochs
results[10][1] = num_rounds
results[11][1] = NUM_CLIENTS * fraction_fit
results[12][1] = NUM_CLIENTS * fraction_evaluate
for key in (history.metrics_centralized):
    results[count][2] = history.metrics_centralized[key][(num_rounds-1):][0][1]
    count += 1
results[9][2] = fl_epochs
results[10][2] = num_rounds
results[11][2] = NUM_CLIENTS * fraction_fit
results[12][2] = NUM_CLIENTS * fraction_evaluate
file = results_path + "history" + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".csv"
results_out = pd.DataFrame(results)
results_out.columns = ['single_server', 'distributed', 'centralized']
results_out.index = ['tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc', 'prc', 'epochs', 'rounds', 'train_clients', 'evaluate_clients']
print(results_out)
pd.DataFrame(results_out).to_csv(file)
print("\nWriting out results.")